In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# System
import os

# Time
import time
import datetime

# Numerical
import numpy as np
import pandas as pd

# NLP
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
# from pywsd.utils import lemmatize_sentence
from gensim.models import Word2Vec

# Preprocessing
from sklearn import preprocessing
from sklearn.utils import class_weight as cw
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup

# Model Selection
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

# Machine Learning Models
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

# Evaluation Metrics
from sklearn import metrics 
from sklearn.metrics import f1_score, accuracy_score,confusion_matrix,classification_report

# Deep Learing Preprocessing - Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical

# Deep Learning Model - Keras
from keras.models import Model
from keras.models import Sequential
from keras.utils import plot_model
# Deep Learning Model - Keras - CNN
from keras.layers import Conv1D, Conv2D, Convolution1D, MaxPooling1D, SeparableConv1D, SpatialDropout1D, \
    GlobalAvgPool1D, GlobalMaxPool1D, GlobalMaxPooling1D 
from keras.layers.pooling import _GlobalPooling1D
from keras.layers import MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.layers import Embedding
from keras.layers import Input, Add, concatenate, Dense, Activation, BatchNormalization, Dropout, Flatten
from keras.layers import LeakyReLU, PReLU, Lambda, Multiply
# Deep Learning Model - Keras - LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional


# Deep Learning Parameters - Keras
from keras.optimizers import RMSprop, Adam

# Deep Learning Callbacs - Keras
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from keras.layers import MaxPooling3D, GlobalMaxPooling3D, GlobalAveragePooling3D
# Visualization


import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
books_neg = pd.read_csv("/content/drive/My Drive/LSTM/books_negative.csv")
books_pos = pd.read_csv("/content/drive/My Drive/LSTM/books_positive.csv")
dvd_neg = pd.read_csv("/content/drive/My Drive/LSTM/dvd_negative.csv")
dvd_pos = pd.read_csv("/content/drive/My Drive/LSTM/dvd_positive.csv")
ele_neg = pd.read_csv("/content/drive/My Drive/LSTM/electronics_negative.csv")
ele_pos = pd.read_csv("/content/drive/My Drive/LSTM/electronics_positive.csv")
kit_neg = pd.read_csv("/content/drive/My Drive/LSTM/kitchen_negative.csv")
kit_pos = pd.read_csv("/content/drive/My Drive/LSTM/kitchen_positive.csv")


In [ ]:
books_neg['label'] = 0 
books_pos['label'] = 1
books = pd.concat([books_neg,books_pos],axis = 0)
books['domain'] = 'books'
dvd_neg['label'] = 0
dvd_pos['label'] = 1
dvd = pd.concat([dvd_pos, dvd_neg],axis = 0)
dvd['domain'] = 'dvd'
ele_neg['label'] = 0
ele_pos['label'] = 1
ele = pd.concat([ele_neg, ele_pos],axis = 0)
ele['domain'] = 'electronics'
kit_neg['label'] = 0
kit_pos['label'] = 1
kit = pd.concat([kit_neg, kit_pos],axis = 0)
kit['domain'] = 'kitchen'
alldata = pd.concat([books,dvd,ele,kit])

In [ ]:
books =books.reset_index(drop=True)
dvd =dvd.reset_index(drop=True)
ele =ele.reset_index(drop=True)
kit =kit.reset_index(drop=True)

In [ ]:
x_books = books['review_text']
y_books = books['label']

x_dvd = dvd['review_text']
y_dvd = dvd['label']

x_ele = ele['review_text']
y_ele = ele['label']

x_kit = kit['review_text']
y_kit = kit['label']

In [ ]:
def clean_doc(doc):
    #split into words
    tokens = word_tokenize(doc)
    #convert to lower case
    tokens = [w.lower() for w in tokens]
    #prepare regex for char filtering
    re_punc =re.compile('[%s]'% re.escape(string.punctuation))
    #remove punctuation
    stripped =[re_punc.sub('',w) for w in tokens]
    #filter out stop words
    #stop_words = set(stopwords.words('english'))
    #words =[w for w in words if not w in stop_words]
    #remove remaining tokens that are not alphabetic
    words = [w for w in stripped if w.isalpha()]
    #filter out short tokens
    tokens = [word for word in words if len(word)>1]
    return tokens

In [ ]:
text_books = []
for m in range(0,len(x_books)):
    X = ''.join(str(i)for i in x_books[m])
    te = clean_doc(X)
    trainx = " ".join(te)
    text_books.append(trainx)

text_dvd = []
for m in range(0,len(x_dvd)):
    X = ''.join(str(i)for i in x_dvd[m])
    te = clean_doc(X)
    trainx = " ".join(te)
    text_dvd.append(trainx)


text_ele = []
for m in range(0,len(x_ele)):
    X = ''.join(str(i)for i in x_ele[m])
    te = clean_doc(X)
    trainx = " ".join(te)
    text_ele.append(trainx)


text_kit = []
for m in range(0,len(x_kit)):
    X = ''.join(str(i)for i in x_kit[m])
    te = clean_doc(X)
    trainx = " ".join(te)
    text_kit.append(trainx)

In [ ]:
#tokens
tokens_b = []
for m in range(0,len(x_books)):
    X = ''.join(str(i)for i in x_books[m])
    te = clean_doc(X)
    tokens_b.append(te)

In [ ]:
#define the word2vec model
w2v_model = Word2Vec(tokens_b,min_count=1,size=100,workers=4)

In [ ]:
#convert text data to vector
x_wv_b=[]
for i in range(len(tokens_b)): 
    vector=w2v_model.wv[tokens_b[i]]
    x_wv_b.append(vector)
    
X_wv_b=np.array(x_wv_b)

In [ ]:
max_len = 120

In [ ]:
# pad the vectors
X_wv_b_padded=pad_sequences(X_wv_b,  maxlen=max_len, padding='post')

In [ ]:
print(X_wv_b.shape)
print(X_wv_b_padded.shape)

(2000,)
(2000, 120, 100)


In [ ]:
#Nerual Embedding Averaging
X_wv_b_average=[]
for i in X_wv_b:
    target_vector=[]
    target_vector=np.mean(i, axis=0)
    X_wv_b_average.append(target_vector) 

In [ ]:
label_encoder = LabelEncoder()

Y_books = label_encoder.fit_transform(y_books)
Y_books = to_categorical(Y_books)
Y_books.shape

(2000, 2)

In [ ]:
#split to validation and training
X_train_b, X_valid_b = X_wv_b_padded[:1700], X_wv_b_padded[1700:]
Y_train_b, Y_valid_b = Y_books[:1700], Y_books[1700:]

In [ ]:
# define the LSTM model
def LSTM_model():
    model = Sequential()
    model.add(LSTM(120,return_sequences=False))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    # summarize defined model
    model.build(input_shape=(None,120,100))
    model.summary()
    plot_model(model, to_file='LSTMmodel.png', show_shapes=True)
    return model

In [ ]:
model_LSTM = LSTM_model()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 120)               106080    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 242       
Total params: 106,322
Trainable params: 106,322
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# fit network
history = model_LSTM.fit(X_train_b, Y_train_b, validation_data=(X_valid_b, Y_valid_b),batch_size=128,epochs=200)

Train on 1700 samples, validate on 300 samples
Epoch 1/200
1700/1700 [==============================] - 6s 4ms/step - loss: 0.4660 - accuracy: 0.7706 - val_loss: 0.9613 - val_accuracy: 0.5433
Epoch 2/200
1700/1700 [==============================] - 6s 3ms/step - loss: 0.4462 - accuracy: 0.7847 - val_loss: 1.3971 - val_accuracy: 0.2867
Epoch 3/200
1700/1700 [==============================] - 6s 3ms/step - loss: 0.4325 - accuracy: 0.7835 - val_loss: 1.3119 - val_accuracy: 0.3400
Epoch 4/200
1700/1700 [==============================] - 6s 3ms/step - loss: 0.4170 - accuracy: 0.7947 - val_loss: 1.2440 - val_accuracy: 0.4400
Epoch 5/200
1700/1700 [==============================] - 6s 4ms/step - loss: 0.4101 - accuracy: 0.8059 - val_loss: 1.2204 - val_accuracy: 0.4900
Epoch 6/200
1700/1700 [==============================] - 6s 3ms/step - loss: 0.3875 - accuracy: 0.8194 - val_loss: 1.2926 - val_accuracy: 0.4900
Epoch 7/200
1700/1700 [==============================] - 6s 4ms/step - loss: 0.3623